# Detectron 2 Face Analyze


Mait Application use detectron2 for face recognizer and image processing

Check Cuda that work or not?

In [1]:
# imports are always needed
import torch


# get index of currently selected device
torch.cuda.current_device() 
torch.cuda.device_count() 
torch.cuda.get_device_name(0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))

Using device: cuda
NVIDIA GeForce RTX 4060 Ti


Import dataset

In [2]:
import os
import json
from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog

def load_coco_json(json_file, image_root, dataset_name):
    with open(json_file) as f:
        coco = json.load(f)
    
    categories = {cat['id']: cat['name'] for cat in coco['categories']}
    
    dataset_dicts = []
    skipped_images = 0  # ตัวนับจำนวนภาพที่ถูกข้าม
    skipped_annotations = 0  # ตัวนับจำนวน annotations ที่ถูกข้าม
    
    for img in coco['images']:
        record = {}
        
        record["file_name"] = os.path.join(image_root, img["file_name"])
        record["image_id"] = img["id"]
        record["height"] = img["height"]
        record["width"] = img["width"]
        
        annos = [anno for anno in coco['annotations'] if anno['image_id'] == img['id']]
        objs = []
        for anno in annos:
            # ตรวจสอบว่า segmentation ไม่ว่างและไม่ใช่ 'iscrowd'
            if not anno.get("segmentation") or anno.get("iscrowd"):
                skipped_annotations += 1
                continue  # ข้าม instance นี้
                
            # ตรวจสอบว่า segmentation เป็น list หรือ dict ตาม COCO format
            segm = anno["segmentation"]
            if isinstance(segm, list):
                # polygons
                if len(segm) == 0:
                    skipped_annotations += 1
                    continue
            elif isinstance(segm, dict):
                # RLE
                if not segm:
                    skipped_annotations += 1
                    continue
            else:
                skipped_annotations += 1
                continue
            
            obj = {
                "bbox": anno["bbox"],
                "bbox_mode": BoxMode.XYWH_ABS,
                "category_id": anno["category_id"],
                "iscrowd": anno["iscrowd"]
            }
            obj["segmentation"] = segm
            objs.append(obj)
        
        if not objs:
            skipped_images += 1  # เพิ่มตัวนับถ้าไม่มี annotations ที่ถูกต้อง
            continue  # ข้าม image นี้ถ้าไม่มี annotations ที่ถูกต้อง
                
        record["annotations"] = objs
        dataset_dicts.append(record)
    
    print(f"Loaded {len(dataset_dicts)} images from {dataset_name}, skipped {skipped_images} images with no valid annotations, skipped {skipped_annotations} annotations.")
    return dataset_dicts

# ลงทะเบียน Dataset
dataset_root = r"C:\Users\nnewr\Documents\Mait_FaceAnalyze\dataset"

train_json = os.path.join(dataset_root, "train", "_annotations.coco.json")
train_images = os.path.join(dataset_root, "train")
DatasetCatalog.register("my_dataset_train", lambda: load_coco_json(train_json, train_images, "my_dataset_train"))
MetadataCatalog.get("my_dataset_train").set(thing_classes=["acne", "blackhead", "nodule", "papule", "pustule", "whitehead"])

valid_json = os.path.join(dataset_root, "valid", "_annotations.coco.json")
valid_images = os.path.join(dataset_root, "valid")
DatasetCatalog.register("my_dataset_val", lambda: load_coco_json(valid_json, valid_images, "my_dataset_val"))
MetadataCatalog.get("my_dataset_val").set(thing_classes=["acne", "blackhead", "nodule", "papule", "pustule", "whitehead"])


namespace(name='my_dataset_val',
          thing_classes=['acne',
                         'blackhead',
                         'nodule',
                         'papule',
                         'pustule',
                         'whitehead'])

Example of import image with annotation

In [3]:
import random
import cv2
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# เลือกชุดข้อมูลที่จะตรวจสอบ เช่น ชุด Train
dataset_dicts = DatasetCatalog.get("my_dataset_train")
metadata = MetadataCatalog.get("my_dataset_train")

for d in random.sample(dataset_dicts, 3):  # แสดง 3 ตัวอย่าง
    img = cv2.imread(d["file_name"])
    if img is None:
        print(f"ไม่พบภาพ: {d['file_name']}")
        continue
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2.imshow("Sample", out.get_image()[:, :, ::-1])
    cv2.waitKey(0)
cv2.destroyAllWindows()


Loaded 678 images from my_dataset_train, skipped 113 images with no valid annotations, skipped 114 annotations.


Configuration an image

In [ ]:
from detectron2.config import get_cfg
from detectron2 import model_zoo
import os
from detectron2.data import transforms as T


# สร้าง Configuration
cfg = get_cfg()

# โหลดการตั้งค่าจาก model zoo สำหรับ Mask R-CNN
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))

# ตั้งค่าชุดข้อมูล
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)

# ตั้งค่า DataLoader
cfg.DATALOADER.NUM_WORKERS = 2

# ใช้น้ำหนักจาก pretrained model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

# ตั้งค่า solver
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 2000  # ปรับตามขนาด dataset ของคุณ

# ตั้งค่า Learning Rate Scheduler
cfg.SOLVER.WARMUP_METHOD = "linear"
cfg.SOLVER.WARMUP_FACTOR = 1.0 / 3
cfg.SOLVER.WARMUP_ITERS = 1000
cfg.SOLVER.STEPS = (1500, 1800)  # ขั้นตอนที่ต้องลด Learning Rate

# ตั้งค่า ROI Heads
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 6  # จำนวนคลาสของคุณ

# เปิดการใช้งาน Mask
cfg.MODEL.MASK_ON = True

# ตั้งค่า output directory
cfg.OUTPUT_DIR = "./output"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# ตั้งค่า Data Augmentation
from detectron2.data import transforms as T

def build_transforms():
   return T.TransformList([
        T.Resize((640, 640)),
        T.RandomFlip(prob=0.5, horizontal=True, vertical=False),
        T.RandomBrightness(0.8, 1.2),
        T.RandomContrast(0.8, 1.2),
    ])

cfg.INPUT.AUGMENTATIONS = build_transforms()

# ตั้งค่า Anchor Sizes และ Aspect Ratios
cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[32, 64, 128, 256, 512]]
cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.5, 1.0, 2.0]]


AttributeError: module 'detectron2.data.transforms' has no attribute 'Compose'

In [5]:
import json

valid_json_path = r"C:\Users\nnewr\Documents\Mait_FaceAnalyze\dataset\valid\_annotations.coco.json"

with open(valid_json_path, 'r') as f:
    coco_val = json.load(f)

# ตรวจสอบว่าทุก annotation มี segmentation ที่ไม่ว่างเปล่า
missing_segmentation = [anno for anno in coco_val['annotations'] if not anno.get('segmentation') or anno.get('iscrowd')]

print(f"Number of annotations missing segmentation: {len(missing_segmentation)}")


Number of annotations missing segmentation: 31


In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
import os

class MyTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, cfg, False, output_folder)

# ใช้ MyTrainer แทน DefaultTrainer
trainer = MyTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

# ประเมินผลหลังการฝึก
evaluator = COCOEvaluator("my_dataset_val", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "my_dataset_val")
inference_on_dataset(trainer.model, val_loader, evaluator)
